**Decision Tree** 

Алгоритм дерева решений попадает под категорию контролируемых алгоритмов обучения. Он работает как для непрерывных, так и для категориальных выходных переменных.
![texst](https://media.geeksforgeeks.org/wp-content/uploads/decisionTree.png)

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
#Импортируем набор данных и определите объект, а также целевые наборы данных / столбцы 
dataset = pd.read_csv('breast.csv',
                      names=['A','B','C','D','E','class'])
dataset = dataset.drop(dataset.columns[[0]], axis=1)

In [23]:
dataset

,B,C,D,E,class
0,10.38,122.80,1001.0,0.11840,0
1,17.77,132.90,1326.0,0.08474,0
2,21.25,130.00,1203.0,0.10960,0
3,20.38,77.58,386.1,0.14250,0
4,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...
564,22.39,142.00,1479.0,0.11100,0
565,28.25,131.20,1261.0,0.09780,0
566,28.08,108.30,858.1,0.08455,0
567,29.33,140.10,1265.0,0.11780,0


In [24]:
def entropy(target_col):
#Рассчитываю энтропию набора данных. target_col указывает целевой столбец
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

#Рассчитываю информационный прирост набора данных.
#data(набор данных) для которого следует рассчитать IG
#split_attribute_name (название объект) 
#target_name(имя целевой функции) 
def InfoGain(data,split_attribute_name,target_name="class"):
    total_entropy = entropy(data[target_name])
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

def decision_tree(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
#data данные, для которых должен быть запущен алгоритм decision_tree;
#originaldata это исходный набор данных, необходимый для расчета значения целевой функции
#features пространство признаков набора данных. 
#target_attribute_name имя целевого атрибута
#parent_node_class Это значение целевой функции режима родительского узла для определенного узла.
  
    #Определяю критерий остановки. Если все target_values имеют одинаковое значение,то возвращаю это значение.
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #Проверка набора данных на пустоту
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #Если пространство объектов пустое, возвращаю значение целевой функции 
    elif len(features) ==0:
        return parent_node_class
    
    #Если ничего не выполняется, то строим дерево.
    else:
        #Значение целевой функции текущего узла
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        #Разделяю набор данных
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        #Корень дерева
        tree = {best_feature:{}}
        
        features = [i for i in features if i != best_feature]
        
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = decision_tree(sub_data,dataset,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
            
        return(tree)    

#Прогнозирование    
def predict(query,tree,default = 1):
#Спускаемся по дереву и проверяем, достигли ли листа или все еще находимся в поддереве.
    
    #если эта функция существует в tree.keys () для первого вызова,то прогнозируем. Если это значение не существует,то возвращаю значение.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #Проверяем на ложную квалификацию
            try:
                result = tree[key][query[key]] 
            except:
                return default
            #Обращаемся к узлу
            result = tree[key][query[key]]
            #Если не достигли корневого узла, то идём глубже по дереву
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
#Проверка прогнозирования
def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 


#Считаю точность прогноза
def test(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')
    
tree = decision_tree(training_data,training_data,training_data.columns[:-1])
pprint(tree)
test(testing_data,tree)

{'B': {10.38: 0.0,
       10.94: 1.0,
       11.79: 1.0,
       12.44: 1.0,
       13.86: 1.0,
       14.26: 0.0,
       14.34: 0.0,
       14.36: 1.0,
       14.63: 1.0,
       14.64: 1.0,
       14.88: 1.0,
       15.05: 0.0,
       15.7: 0.0,
       15.71: 1.0,
       15.79: 0.0,
       16.15: 0.0,
       16.34: 1.0,
       16.4: 0.0,
       16.49: 1.0,
       16.52: 1.0,
       16.84: 1.0,
       17.33: 1.0,
       17.6: 0.0,
       17.77: 0.0,
       17.88: 0.0,
       17.89: 0.0,
       18.0: 1.0,
       18.24: 1.0,
       18.42: 1.0,
       18.57: 0.0,
       18.66: 0.0,
       18.7: 0.0,
       18.75: 1.0,
       19.04: 1.0,
       19.31: 1.0,
       19.65: 0.0,
       19.98: 0.0,
       20.13: 0.0,
       20.25: 0.0,
       20.28: 0.0,
       20.38: 0.0,
       20.68: 0.0,
       20.82: 0.0,
       20.83: 0.0,
       20.98: 1.0,
       21.01: 1.0,
       21.25: 0.0,
       21.31: 0.0,
       21.35: 0.0,
       21.38: 0.0,
       21.53: 0.0,
       21.58: 0.0,
       21.59: 0.0

Мой алгоритм даёт результат на ~20% хуже, чем в sklearn.

In [25]:
from sklearn import tree
from sklearn import metrics
df = pd.read_csv('breast1.csv', header = 0)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df.iloc[450:, :-1], df.iloc[450:, -1:])
y_test = df['diagnosis']
df = df.drop(df.columns[[-1]], axis=1)
df = df.to_numpy()
X = df
y_pred = clf.predict(X)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9191564147627417
